In [20]:
import numpy as np
import pandas as pd
import mysql.connector
import requests
from bs4 import BeautifulSoup
from flashtext.keyword import KeywordProcessor
from nltk import tokenize
from nltk.corpus import stopwords

In [2]:
#Managing given keywords
with open('Interest groups - 5th Sept.txt','r') as f:
    c=f.readlines()

In [3]:
c[0]=c[0][3:]

In [4]:
#len(c)


In [5]:
keywords=np.array(c)

In [6]:
for i in range(len(keywords)):
    if i == len(keywords)-1:
        keywords[i]=keywords[i]
    else:
        keywords[i]=keywords[i][:-1]

In [7]:
#loads All sites file
with open('Quantcast-Top-Million.csv.txt','r') as f:
    sites=f.readlines()

In [8]:
weblinks=[]
for i in range(6000,7100):
    j=sites[i][5:-1]
    if j=='Hidden profile':
        pass
    else:
        weblinks.append(j)

In [9]:
for i in range(len(weblinks)):
    weblinks[i]='https://www.'+weblinks[i]

In [10]:
#Sample
weblinks[:20]

['https://www.indiamart.com',
 'https://www.booksamillion.com',
 'https://www.edweek.org',
 'https://www.omgsweeps.info',
 'https://www.perksatwork.com',
 'https://www.twinspires.com',
 'https://www.cdkeys.com',
 'https://www.ti.com',
 'https://www.bjsrestaurants.com',
 'https://www.diariogol.com',
 'https://www.pearsonrealize.com',
 'https://www.consumerlab.com',
 'https://www.mandtbank.com',
 'https://www.subscene.com',
 'https://www.immihelp.com',
 'https://www.peoplem.ag',
 'https://www.zipjobs.com',
 'https://www.premierguitar.com',
 'https://www.mrmoneymustache.com',
 'https://www.lync.com']

In [ ]:
#Saving links in database

conn= mysql.connector.connect(host='localhost',user='sachin',passwd='mysqlsachin',database='interntask2')

mycursor=conn.cursor()

mycursor.execute("""DROP TABLE IF EXISTS weblink""")
mycursor.execute("CREATE TABLE weblinks(weblinks VARCHAR(800))")




query = "INSERT INTO randomlinks (weblinks) VALUES (%s)"
mycursor.executemany(query, [(list_item, ) for list_item in weblinks])

conn.commit()


In [ ]:
len(weblinks)

In [11]:
kp0=KeywordProcessor()

for word in keywords:
    kp0.add_keyword(word)

In [150]:
import nltk
from bs4.element import Comment

In [151]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [154]:
def gethtmlcontent(url):
    try:
        soup = BeautifulSoup(url, 'html.parser')  #Parse html code
        texts = soup.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        return u" ".join(t.strip() for t in visible_texts)
        
    
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []
    

In [155]:
sw=stopwords.words('english')
to_remove=['[]','','1','()','||','=','.',',','\n',':',';','\\','//','/']

for i in to_remove:
    sw.append(i)

In [156]:
def top_keywords(text):
    x=str(text)
    
    x=x.split(' ')
    words=[w for w in x]
    sws=set(sw)
    useful_words=[w for w in words if w not in sws]
    
    uniq=np.unique(useful_words,return_counts=True)
    
    if len(uniq[1])==0:
        return 0
    else:
        index=np.argmax(uniq[1])
    
    
    return uniq[0][index]
    

In [186]:
def find_class(text_from_html):
    x=str(text_from_html)
    y0 = len(kp0.extract_keywords(x))
    if y0==0:
        top_kw=top_keywords(text_from_html)
        #Category='None'
        return 0,top_kw
    
    else:
        kw=kp0.extract_keywords(x)
        uniq_kw=np.unique(kw,return_counts=True)
        
        m=np.argmax(uniq_kw[1])
        max_kw=uniq_kw[1][m]
        sum_kw=np.sum(uniq_kw[1])
        
        return (max_kw/(sum_kw))*100,kw,max_kw


In [187]:
sites=['https://palladiumdancecompany.com','https://www.beatport.com/genre/dance/39','https://en.wikipedia.org/wiki/Dance','https://en.wikipedia.org/wiki/Music','https://support.apple.com/music','https://www.theguardian.com/music']

In [188]:
len(sites)

6

In [ ]:
confidences=[]
category=[]
links=[]
combine=[]
for i in range(len(sites)):
    url= sites[i]
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
    

    try:
        page = requests.get(url,headers=headers)  #to extract page from website
    except requests.exceptions.ConnectionError:
    
        i+=1
        continue
    sc=page.status_code
    if sc == 403:
        continue
    
    html_code = page.content      #to extract html code from page
    text_from_html=gethtmlcontent(html_code)
    #print(text_from_html)
    confidence,kw,number_of_time_repeated=find_class(text_from_html)
    confidences.append(confidence)
    
    print("Confidience : ",confidence)
    uniqkw=np.unique(kw,return_counts=True)
    
    m=np.argmax(uniqkw[1])
    links.append(sites[i])
    
    print("Top_keyword: ",uniqkw[0][m])
    
    print("Link: ",sites[i])
    category.append(uniqkw[0][m])
    combine.append((sites[i],confidence,uniqkw[0][m]))
    print('-'*75)
    
    

In [ ]:
conn= mysql.connector.connect(host='localhost',user='sachin',passwd='mysqlsachin',database='atginterntask2')

mycursor=conn.cursor()

mycursor.execute("""DROP TABLE IF EXISTS web_classification""")
mycursor.execute("CREATE TABLE web_classification(weblinks VARCHAR(800),confidence INT(200),kw VARCHAR(600))")


query1 = """INSERT INTO  web_classification(weblinks) 
           VALUES (%s)"""

#mycursor.executemany((query1,), category)
mycursor.executemany(query1, [(list_item, ) for list_item in links])
# mycursor.executemany((query2,), combined.any())
# mycursor.executemany((query3,), combined.any())
                     
conn.commit()